<a href="https://colab.research.google.com/github/PhilChodrow/PIC16B/blob/master/lectures/tf/tf-5-live.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Generation with Recurrent Neural Networks

In this set of lecture notes, we'll consider a new kind of machine learning task. Previously, we've focused on *classification* problems. In classification problems, the goal is to assign a given piece of data to one of several categories. Today, we'll instead consider a simple  *generation* problem. A *generative* model can be used to create "realistic" examples after it's been trained. Generative models are at the heart of machine learning topics like [deepfakes](https://en.wikipedia.org/wiki/Deepfake), [language generation](https://aiweirdness.com/post/140219420017/the-silicon-gourmet-training-a-neural-network-to), and [style transfer](https://www.tensorflow.org/tutorials/generative/style_transfer).  

*Parts of these lecture notes were based on [this tutorial](https://keras.io/examples/generative/lstm_character_level_text_generation/). It is recommended to run the code contained in these notes in a Google Colab instance with GPU acceleration enabled.* 

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
# link to Google Drive to read in trained model
from google.colab import drive
drive.mount('/content/drive')


## Our Task

Today, we are going to see whether we can teach an algorithm to understand and reproduce the pinnacle of cultural achievement; the benchmark against which all art is to be judged; the mirror that reveals to humany its truest self. I speak, of course, of *Star Trek: Deep Space Nine.*

<figure class="image" style="width:300px">
  <img src="https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/_images/DS9.jpg" alt="">
  <figcaption><i></i></figcaption>
</figure>

In particular, we are going to attempt to teach a neural  network to generate *episode scripts*. This a text generation task: after training, our hope is that our model will be able to create scripts that are reasonably realistic in their appearance. 


In [ ]:
## miscellaneous data cleaning

start_episode = 20 # Start in Season 2, Season 1 is not very good
num_episodes = 50  # only pick this many episodes to train on

url = "https://github.com/PhilChodrow/PIC16B/blob/master/datasets/star_trek_scripts.json?raw=true"
star_trek_scripts = pd.read_json(url)

cleaned = star_trek_scripts["DS9"].str.replace("\n\n\n\n\n\nThe Deep Space Nine Transcripts -", "")
cleaned = cleaned.str.split("\n\n\n\n\n\n\n").str.get(-2)
text = "\n\n".join(cleaned[start_episode:(start_episode + num_episodes)])
for char in ['\xa0', 'à', 'é', "}", "{"]:
    text = text.replace(char, "")

The result is one long string containing the scripts of 50 episodes of Star Trek: Deep Space 9. How glorious!

Our first step, as usual, is data preparation. What we need to do is format the data in such a way that we can treat the situation as a classification problem after all. That is: 

> Given a string of text, predict the next character in that string. 

Doing this repeatedly will allow the model to generate large bodies of text. 

To do this, we want to split our data like so: 

```
predictor = "to boldly g"
target    = "o"
```

The following function will do this for us. The `max_len` argument gives the number of characters that should be in the predictor string, and the `step_size` argument lets us skip indices if we want to in order to decrease the size of the data. 

Our next step is to vectorize the characters. This is similar to the word vectorization task, but it's simple enough in this case that's arguably more convenient to actually handle it outside of TensorFlow. It is also possible to handle vectorization using TensorFlow tools, as demonstrated in [this tutorial](https://www.tensorflow.org/tutorials/text/text_generation). 

In [ ]:
# vectorization code

chars = sorted(set(text))
char_indices = {char : chars.index(char) for char in chars}
X = np.zeros((len(lines), max_len, len(chars)))
y = np.zeros((len(lines), 1), dtype = np.int32)
for i, line in enumerate(lines):
	for t, char in enumerate(line):
		X[i, t, char_indices[char]] = 1
	y[i] = char_indices[next_chars[i]]

Let's take a look at the results. 

What does this matrix say? `X[0]` is a single sequence of 20 characters. The first row says that the first character of this sequence is the character corresponding to integer `1`. The second row says that the second character corresponds to integer `1`, and so on (`1` is the space character `" "`, in case you're wondering).  

This says that the 21st character (the character after the first 20 characters of the string) is the character with index `42`. 

Now we're ready to perform a train-test split: 

## Modeling

Model time! We'll use a simple *Long Short-Term Memory* (LSTM) model for this example. LSTMs are one example of *recurrent* neural network layers. Here's a diagram illustrating the schematic functioning of a recurrent layer. 

![](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)
*Image credit: [Chris Olah](https://colah.github.io/posts/2015-08-Understanding-LSTMs/), OpenAI*

On the lefthand side, we have a "zoomed out" picture of a recurrent neural network layer. On the righthand side, we see the "zoomed in" version. The key point here is that output $h_2$ depends not only on input $x_2$, but also, indirectly, on inputs $x_0$ and $x_1$. This means that recurrent neural networks are highly suitable for modeling processes that have temporal structure. Text is an example: the last few characters are the "history" of the text. Timeseries data are another clear example, and indeed, we can use a very similar workflow to the one we'll use today in order to do forecasting in timeseries. 

Since training for this kind of task gets expensive fast, we'll use just one LSTM layer followed by a `Dense` output layer. 

Time for training. We'll do just one epoch for now, mostly just to prove that we've set up our model correctly. 

In [ ]:
# code I used to train and save the model
# model.fit(X_train, 
#           y_train,
#           validation_data= (X_val, y_val),
#           batch_size=128, epochs = 200)
# model.save('/content/drive/MyDrive/DS9_model') 



Rather than training the entire model live during lecture, I'm instead going to load in a saved model that I previously trained for 200 epochs on Google Colab. On Colab, each epoch takes around 10s or so. 200 epochs corresponds to roughly 30 minutes. 

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/DS9_model')

Generative models define *probability distributions* over the space of possible outputs. So, our overall algorithm is going to generate new text in a partially randomized way. To make this happen, we define a `sample` function which will take the model outputs, turn them into probabilities, and then sample from the probabilities to produce a single character (well, technically, an integer corresponding to a single character). 

An important parameter here is the so-called *temperature* (this terminology comes from statistical physics. When the temperature is high, the model will more frequently choose low-probability characters. This is sometimes interpreted as "creativity," and leads to more unpredictable outputs. When the temperature is low, on the other hand, the model will "play it safe" and tend to stick to known patterns. In the extreme limiting case as the temperature approaches 0, the model will ultimately get stuck in "loops" in which it repeats common phrases over and over again. 

I'm not going to dwell on the math here, but if you're familiar with phrases like "softmax" or "Boltzmann distribution," this code implements and samples from such a distribution using the model predictions.

Note that this function takes in some model predictions and returns a single integer, which we can interpret as a character. 

Now that we know how to sample from the model predictions to create a new character, let's now define a convenient function that will allow us to create entire strings of specified length using this process. There's some index management here that can be a little tricky. 

Let's try it out! We'll create strings of length 500, separating the real seed text from the generated text. We'll also vary the temperature parameter of the `sample()` function, which controls how random the model's predictions can be. 

Let's make a few observations. 

1. First of all, it can take a surprisingly long time to make predictions using our model. This is because we have to call the `predict()` method *for each character*, in order to ensure that the model appropriately takes into account its recent predictions. This can take a pretty long time! 
2. Second, determining a good value for the temperature can take some experimentation. Note that low temperatures don't necessarily correspond to "more realistic" text -- they just correspond to highlighting common patterns in the text, possibly in excess. Higher temperatures also don't necessarily correspond to a "creative" algorithm in any normal sense of the word -- set the temperature too high, and you'll just get gibberish. 

## Specialization

In this case, we were able to create a model for generating Star Trek scripts using an instance of Google Colab in roughly 30 minutes. This model is highly limited. Although it clearly has learned some relevant features of Star Trek scripts, there's no way that you'd mistake the output of the model for an actual script by a screenwriter. Considering how hard this was, imagine how much effort and computational resources are required to create more general language models! Indeed, as highlighted in a [recent and controversial paper](https://faculty.washington.edu/ebender/papers/Stochastic_Parrots.pdf), training large language models in this day and age can require energy expenditure comparable to a trans-Atlantic flight! 